In [14]:
# R Notebook on Krafttier
# http://localhost:8888/?token=2db7380f0b69da12b02f8d4ee46e03a2487032237ef536c7

In [15]:
#Libraries
suppressPackageStartupMessages({
    library(DESeq2, quietly=TRUE)
    })

#Global option for image size
options(repr.plot.width = 25, repr.plot.height = 13)

#Global Options for how to display data.frame
options(repr.matrix.max.cols=20, repr.matrix.max.rows=200)

#Plot sizing
options(repr.plot.width = 10, repr.plot.height = 5, repr.plot.res = 150)

In [16]:
outdir = "/mnt/mega/magdean/testing/output"

coldata_path = file.path("/mnt/mega/magdean/testing/coldata_sim.tsv")
counts_path = file.path("/mnt/mega/magdean/testing/counts_sim.tsv")
feature_to_mag_path = file.path("/mnt/mega/magdean/testing/feature_to_mag_sim.tsv")
reference_level = "normal" # The base condition to compare against

In [17]:
# normalize_counts = function(count_matrix, dds_raw, coldata) {
#     #' Normalizes counts using DESeq2
#     #' 
#     #' @param count_matrix A count matrix with genes as rows and samples as columns
#     #' @param dds_raw A DESeqDataSet object
#     #' @param coldata A data.frame with first column 'sample' and second column 'condition'
#     #' We use fitType mean because it always works and we are not interested
#     #' in the fit anyways, only in the normalization.
#     #' 
#     #' @return A normalized count matrix
#     #' 
#     fit_type = 'mean'
    
#     dds = DESeq(dds_raw, fitType=fit_type, quiet=FALSE)
#     normalizationMatrix <- rep(dds@colData@listData$sizeFactor, each=nrow(count_matrix))
#     normalizedCountMatrix <- count_matrix / normalizationMatrix

#     return(normalizedCountMatrix)
# }

In [18]:
# default_normalization = function(count_matrix, coldata_path, reference_level) {
#     #' Normalizes counts with DESeq2 using a simple design formula of ~condition
#     #' 
#     #' @param count_data_path A path to a count matrix
#     #' @param coldata_path A path to a coldata file
#     #' @param reference_level The reference level for the condition
#     #' 
#     #' @return A normalized count matrix
    
#     count_matrix = round(count_matrix)

#     # Load column data
#     coldata <- read.csv(file=coldata_path, sep='\t', row.names=1)
#     coldata[,1] <- factor(coldata[,1], levels=unique(coldata[,1]))
#     coldata[,1] <- relevel(coldata[,1], ref=reference_level)

#     # Assuming the first column of coldata is 'condition'
#     names(coldata)[1] <- 'condition'

#     # Create DESeq2 object
#     dds_raw = DESeqDataSetFromMatrix(countData=count_matrix,
#                                      colData=coldata,
#                                      design=~condition)

#     # Ensure levels and reference level are set correctly
#     condition_levels = unique(coldata[,1])
#     dds_raw$condition = factor(dds_raw$condition, levels=condition_levels)
#     dds_raw$condition = relevel(dds_raw$condition, ref=reference_level)

#     # Normalize counts
#     normalized_counts = normalize_counts(count_matrix, dds_raw, coldata)

#     return(normalized_counts)
# }


In [19]:
default_normalization <- function(count_matrix, coldata_path, reference_level) {
    #' Normalizes counts with DESeq2 using a simple design formula of ~condition
    #' 
    #' @param count_data_path A path to a count matrix
    #' @param coldata_path A path to a coldata file
    #' @param reference_level The reference level for the condition
    #' 
    #' @return A normalized count matrix

    # Load column data
    coldata = read.csv(file=coldata_path, sep='\t', row.names=1)
    coldata[,1] = factor(coldata[,1], levels=unique(coldata[,1]))
    coldata[,1] = relevel(coldata[,1], ref=reference_level)

    # Assuming the first column of coldata is 'condition'
    names(coldata)[1] <- 'condition'

    # Create DESeq2 object
    dds_raw = DESeqDataSetFromMatrix(countData = as.matrix(round(count_matrix)),
                                     colData = coldata,
                                     design = ~ condition)

    # Perform size factor estimation
    dds_norm = estimateSizeFactors(dds_raw)
    
    # Extract normalized counts
    normalized_counts = counts(dds_norm, normalized=TRUE)

    return(normalized_counts)
}


In [20]:
default_deseq_without_normalization <- function(normalized_count_matrix, coldata_path, reference_level) {
    #' Runs with DESeq2 using a simple design formula of ~condition
    #' Circumvents the usual DESeq2 normalization.#
    #' 
    #' @param normalized_count_matrix A normalized count matrix
    #' @param coldata_path A path to a coldata file
    #' @param reference_level The reference level for the condition
    #'  
    #' @return A DESeq2 object
    
    # Load column data
    coldata = read.csv(file=coldata_path, sep='\t', row.names=1)
    coldata[,1] = factor(coldata[,1], levels=unique(coldata[,1]))
    coldata[,1] = relevel(coldata[,1], ref=reference_level)

    # Assuming the first column of coldata is 'condition'
    names(coldata)[1] <- 'condition'

    # Create DESeq2 object
    dds_raw = DESeqDataSetFromMatrix(countData = as.matrix(round(normalized_count_matrix)),
                                     colData = coldata,
                                     design = ~ condition)
    dds_raw$condition = relevel(dds_raw$condition, ref=reference_level)

    # Create a matrix with 1s as normalization factors and use them to overwrite
    # the normalization factors in the DESeq2 object
    normalization_factors = matrix(1,
                                   ncol=ncol(normalized_count_matrix),
                                   nrow=nrow(normalized_count_matrix))
    normalizationFactors(dds_raw) = normalization_factors

    # Run DESeq2
    dds = DESeq(dds_raw)
    
    return(dds)
}

In [21]:
default_shrunken_results <- function(dds, reference_level, shrinkage_type='apeglm') {
    #' Computes shrunken results for all pairwise comparisons against a reference level
    #'
    #' @param dds A DESeq2 object
    #' @param reference_level The reference level for the condition
    #' @param shrinkage_type "apeglm", "ashr", or "normal"
    #'
    #' @return A list of DESeq2 results for each comparison
    
    # Check if the shrinkage type is valid
    if(!(shrinkage_type %in% c("apeglm", "ashr", "normal"))) {
        stop("Invalid shrinkage type. Must be one of 'apeglm', 'ashr', or 'normal'.")
    }

    # Extract the contrasts (need to remove "Intercept")
    contrast_names = resultsNames(dds)
    contrast_names = setdiff(contrast_names, "Intercept")
    print(contrast_names)

    # Iterate over comparison levels and compute shrunken log2 fold changes
    results_list <- list()
    for(contrast in contrast_names) {
        res_shrunk <- lfcShrink(dds, coef=contrast, type=shrinkage_type)
        results_list[[contrast]] <- res_shrunk
    }

    return(results_list)
}

In [22]:
write_normalized_counts_to_tsv <- function(normalized_counts,
                                           feature_to_mag,
                                           directory=outdir) {
    #' Writes a normalized count matrix to a TSV file, including MAG information
    #' 
    #' @param normalized_counts A normalized count matrix with features as rows
    #' @param feature_to_mag A data frame mapping features to MAGs
    #' @param directory The directory to write the TSV file to
    #' 
    #' @return The path to the written TSV file
    
    # Create the directory if it doesn't exist
    if(!dir.exists(directory)) {
        dir.create(directory, recursive = TRUE)
    }
    
    # Ensure the row names of normalized_counts are used as a feature_id column
    normalized_counts_df <- data.frame(feature_id=rownames(normalized_counts), normalized_counts)
    
    # Ensure feature_to_mag has 'feature_id' as the first column if it's not already
    if(!"feature_id" %in% names(feature_to_mag)) {
        feature_to_mag <- data.frame(feature_id=rownames(feature_to_mag), feature_to_mag)
    }

    # Find the matching MAG names for each feature_id in the order of normalized_counts_df's rows
    mag_names <- feature_to_mag$mag_id[match(normalized_counts_df$feature_id, feature_to_mag$feature_id)]

    # Add the MAG names as a new column to the normalized_counts_df
    normalized_counts_df$mag_id <- mag_names

    # Define the path for the normalized counts TSV file
    normalized_counts_path = file.path(directory, "normalized_counts.tsv")

    # Write the data frame, now including MAG information, to a TSV file
    write.table(normalized_counts_df, file=normalized_counts_path, sep="\t", quote=FALSE, row.names=FALSE)
    
    return(normalized_counts_path)
}


In [23]:
write_shrunken_results_to_tsv <- function(shrunken_results, directory=outdir) {
  #' Writes results to .tsv files.
  #' 
  #' @param shrunken_results A list of DESeq2 results objects
  #' @param directory The directory to write the files to
  #'  
  #' @return NULL

  # Create the directory if it doesn't exist. Otherwise throw an error message
  if(!dir.exists(directory)) {
    dir.create(directory)
  }
  
  # Iterate over the shrunken results list
  for(contrast_name in names(shrunken_results)) {
    result = shrunken_results[[contrast_name]]
    
    # Define the file path, using the contrast name in the file name
    file_path = file.path(directory, paste0(contrast_name, "_results.tsv"))
    
    # Write to file
    write.table(as.data.frame(result), file=file_path, row.names=TRUE, sep="\t", quote=FALSE)
  }
}


In [24]:
main <- function() {
  #' Main function to run the entire pipeline
  #' 
  #' @return NULL

  feature_to_mag = read.csv(file=feature_to_mag_path, sep='\t', row.names=1)
  mag_ids = unique(feature_to_mag$mag_id)
  counts = as.matrix(read.csv(file=counts_path, sep='\t', row.names=1))
  # Ensure mag_id column is correctly treated as a factor
  feature_to_mag$mag_id <- factor(feature_to_mag$mag_id)

  # Create a list to store counts split by mag_id
  mag_counts <- list()

  # Iterate over each mag_id and subset counts accordingly
  for(mag_id in levels(feature_to_mag$mag_id)) {
      mag_rows <- rownames(feature_to_mag[feature_to_mag$mag_id == mag_id, , drop=FALSE])
      if(length(mag_rows) > 0 && all(mag_rows %in% rownames(counts))) {
        mag_counts[[as.character(mag_id)]] <- counts[mag_rows, ]
      }
  }

  # Calculate normalized count matrices for each mag_id
  normalized_mag_counts <- lapply(mag_counts, function(count_matrix) {
      default_normalization(count_matrix, coldata_path, reference_level)
  })

  # Combine all the normalized count matrices into one big matrix
  combined_matrix <- do.call(rbind, normalized_mag_counts)

  # Write the normalized counts to disk
  write_normalized_counts_to_tsv(combined_matrix, feature_to_mag)

  # Run DESeq2 without normalization
  dds = default_deseq_without_normalization(combined_matrix, coldata_path, reference_level)

  # Compute results with lfcShrink
  shrunken_results = default_shrunken_results(dds, reference_level)

  # Write tables to disk
  write_shrunken_results_to_tsv(shrunken_results)
}

In [25]:
main()

converting counts to integer mode

converting counts to integer mode

converting counts to integer mode

converting counts to integer mode

converting counts to integer mode

converting counts to integer mode

using pre-existing normalization factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "condition_speiseeis_vs_normal" "condition_fasting_vs_normal"  


using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



# Sanity Stuff

In [26]:
shrunken_results = default_shrunken_results(dds, reference_level)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'mcols': object 'dds' not found


In [ ]:
print(shrunken_results)

ERROR: Error in print(shrunken_results): object 'shrunken_results' not found


In [ ]:
res <- results(dds, contrast=c("condition", "fasting", "normal"))

In [ ]:
shrunk = lfcShrink(dds, coef="condition_fasting_vs_normal", type="apeglm")

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



In [ ]:
head(normalized_mag_counts$horse)

,reactor_a_replicate_1,reactor_a_replicate_2,reactor_b_replicate_1,reactor_b_replicate_2,reactor_b_replicate_3,reactor_c_replicate_1,reactor_c_replicate_2
horse_0,78.37895,70.86068,62.40489,72.17661,126.88356,104.37127,89.72397
horse_1,20.74028,24.95426,30.97469,24.21114,30.01547,26.09282,12.81771
horse_2,13.50530,19.53966,18.22041,16.44530,20.46509,26.09282,12.81771
horse_3,109.48937,92.28368,91.10203,96.38775,48.20666,65.23204,89.72397
horse_4,17.12279,18.12715,22.32000,22.38389,21.37465,26.09282,25.63542
horse_5,64.39132,77.21696,382.17303,287.33600,398.38710,117.41768,102.54168


In [ ]:
head(combined_matrix)
tail(combined_matrix)

,reactor_a_replicate_1,reactor_a_replicate_2,reactor_b_replicate_1,reactor_b_replicate_2,reactor_b_replicate_3,reactor_c_replicate_1,reactor_c_replicate_2
horse_0,78.37895,70.86068,62.40489,72.17661,126.88356,104.37127,89.72397
horse_1,20.74028,24.95426,30.97469,24.21114,30.01547,26.09282,12.81771
horse_2,13.50530,19.53966,18.22041,16.44530,20.46509,26.09282,12.81771
horse_3,109.48937,92.28368,91.10203,96.38775,48.20666,65.23204,89.72397
horse_4,17.12279,18.12715,22.32000,22.38389,21.37465,26.09282,25.63542
horse_5,64.39132,77.21696,382.17303,287.33600,398.38710,117.41768,102.54168


,reactor_a_replicate_1,reactor_a_replicate_2,reactor_b_replicate_1,reactor_b_replicate_2,reactor_b_replicate_3,reactor_c_replicate_1,reactor_c_replicate_2
zebra_995,55.89580,40.74944,83.48137,101.74053,35.54287,94.81016,43.04519
zebra_996,92.83563,118.45766,55.06218,48.57881,64.86574,108.68482,67.96609
zebra_997,108.87530,166.78839,87.03376,154.90225,147.50291,127.18436,104.21467
zebra_998,120.05446,91.92314,147.42454,113.65609,110.18290,76.31062,79.29377
zebra_999,65.61681,39.32794,36.41209,32.08035,71.08574,76.31062,77.02823
zebra_1000,210.45984,130.77726,102.13146,107.24002,101.29718,247.43139,97.41806


In [ ]:
normalized = default_normalization(counts_path, coldata_path, reference_level)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'ncol': error in evaluating the argument 'x' in selecting a method for function 'as.matrix': non-numeric argument to mathematical function


In [ ]:
coldata <- read.csv(file=coldata_path, sep='\t', row.names=1)
coldata[,1] <- factor(coldata[,1], levels=unique(coldata[,1])) # Make sure this is a factor
counts = as.matrix(read.csv(file=counts_path, sep='\t', row.names=1))
feature_to_mag = read.csv(file=feature_to_mag_path, sep='\t', row.names=1)

In [ ]:
mag_counts <- lapply(names(mag_ids), function(mag_id) {
    mag_rows <- feature_to_mag[feature_to_mag$mag_id == mag_id, "feature_id"]
    mag_counts <- counts[mag_rows, ]
    rownames(mag_counts) <- mag_rows
    mag_counts
})


In [ ]:
mag_counts

list()

In [ ]:
for (mag_id in names(mag_counts)) {
  cat("Matrix for mag_id:", mag_id, "\n")
  print(mag_counts[[mag_id]])
  cat("\n")
}


In [ ]:
# Accessing a specific matrix from mag_counts
specific_matrix <- mag_counts[["matrix_name"]]


In [ ]:
print(mag_counts["horse"])

$horse
   [1]  650.885990  246.785919  500.309425  220.179549  766.585092  744.263222
   [7]  461.500665  279.496705  746.274631 1061.509673  968.283985  329.118106
  [13]  355.472509  981.865404  273.449098   74.078638  413.640079  469.038006
  [19]  854.825542 1112.572618  462.989986  419.340304   14.076223  364.268085
  [25]  505.220426    7.847395  808.075632  126.302853  383.652618  339.257030
  [31]  581.716787  946.600952  420.822933    7.262585  296.424861  273.880173
  [37]  492.156313  309.409167  648.687583 1050.357001  106.160507  439.320565
  [43]  837.649691  552.584592 1133.633919  562.828954  920.751027   94.065753
  [49]  260.296987  656.344333  393.528225  838.616883  147.948440 1244.130995
  [55]  584.085504 4202.937354  484.749656   52.115084 1187.514802  689.195832
  [61]  322.724409  713.002399  249.510154  197.899828  831.617590  795.920558
  [67]  166.724565  630.591590  646.544837  159.015186  300.254314  252.578516
  [73]  745.104412  424.514749  465.510689 44

In [ ]:
for (mag_id in names(mag_counts)) {
  cat("Matrix for mag_id:", mag_id, "\n")
  print(mag_counts[[mag_id]])
  cat("\n")
}


Matrix for mag_id: horse 
   [1]  650.885990  246.785919  500.309425  220.179549  766.585092  744.263222
   [7]  461.500665  279.496705  746.274631 1061.509673  968.283985  329.118106
  [13]  355.472509  981.865404  273.449098   74.078638  413.640079  469.038006
  [19]  854.825542 1112.572618  462.989986  419.340304   14.076223  364.268085
  [25]  505.220426    7.847395  808.075632  126.302853  383.652618  339.257030
  [31]  581.716787  946.600952  420.822933    7.262585  296.424861  273.880173
  [37]  492.156313  309.409167  648.687583 1050.357001  106.160507  439.320565
  [43]  837.649691  552.584592 1133.633919  562.828954  920.751027   94.065753
  [49]  260.296987  656.344333  393.528225  838.616883  147.948440 1244.130995
  [55]  584.085504 4202.937354  484.749656   52.115084 1187.514802  689.195832
  [61]  322.724409  713.002399  249.510154  197.899828  831.617590  795.920558
  [67]  166.724565  630.591590  646.544837  159.015186  300.254314  252.578516
  [73]  745.104412  424.51

In [ ]:
head(feature_to_mag,3)

,mag_id
,<chr>
zebra_0,zebra
zebra_1,zebra
zebra_2,zebra


In [ ]:
head(counts,3)

,reactor_a_replicate_1,reactor_a_replicate_2,reactor_b_replicate_1,reactor_b_replicate_2,reactor_b_replicate_3,reactor_c_replicate_1,reactor_c_replicate_2
zebra_0,695.257898,512.724244,1215.667773,871.440889,1018.350602,1302.802262,1078.930228
zebra_1,876.947635,1136.491008,1237.665338,786.965227,778.793773,914.024768,687.198934
zebra_2,1.463164,2.253887,2.201316,1.675993,1.932696,1.913809,1.453623


In [ ]:
type(counts)

[1] "double"